In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.webdriver import WebDriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd 
import re
import datetime
from datetime import datetime
import requests
import pymongo

In [30]:
# match links

def getDataFromPage(url):

    op = webdriver.ChromeOptions()
    op.add_argument('headless')

    driver = webdriver.Chrome(options=op)
    driver.get(url)

    page_source = driver.page_source

    driver.quit()

    return page_source

def getDataMain(page_source):
    links_from_table = re.findall(r'data-stat="match_report"><([^<]+)', page_source)

    cleaned_list = [item for item in links_from_table if '/td>' not in item and 'Head-to-Head' not in item]
    cleaned_list_final = ['https://fbref.com/' + item.replace('a href="', '').replace('">Match Report', '') for item in cleaned_list]

    return cleaned_list_final


def matchesLinks(url):

    data = getDataFromPage(url)
    list_of_links = getDataMain(data)

    return list_of_links

In [31]:
# replacment 

def get_replacement(club_name):

    replacments_club_names = {
            'Arsenal' : 'Arsenal',
            'Aston Villa' : 'Aston Villa',
            'Bournemouth' : 'Bournemouth',
            'Brentford' : 'Brentford',
            'Brighton' : 'Brighton',
            'Brighton &amp; Hove Albion' : 'Brighton',
            'Burnley' : 'Burnley',
            'Chelsea' : 'Chelsea',
            'Crystal Palace' : 'Crystal Palace',
            'Everton' : 'Everton',
            'Fulham' : 'Fulham',
            'Liverpool' : 'Liverpool',
            'Luton Town' : 'Luton',
            'Manchester City' : 'Man City',
            'Manchester Utd' : 'Man United',
            'Manchester United' : 'Man United',
            'Newcastle Utd' : 'Newcastle',
            'Newcastle United' : 'Newcastle',
            "Nott'ham Forest" : 'Forest',
            'Nottingham Forest' : 'Forest',
            'Sheffield Utd ' : 'Sheffield United',
            'Sheffield United' : 'Sheffield United',
            'Tottenham' : 'Tottenham',
            'Tottenham Hotspur' : 'Tottenham',
            'West Ham' : 'West Ham',
            'West Ham United' : 'West Ham',
            'Wolves' : 'Wolves',
            'Wolverhampton Wanderers' : 'Wolves',
            'Leicester' : 'Leicester',
            'Leicester City' : 'Leicester',
            'Leeds' : 'Leeds',
            'Leeds United' : 'Leeds',
            'Southampton' : 'Southampton',
            'Watford' : 'Watford',
            'Norwich' : 'Norwich',
            'Norwich City' : 'Norwich',
            'West Brom' : 'West Brom',
            'West Bromwich Albion' : 'West Brom',
            'Stoke' : 'Stoke',
            'Stoke City' : 'Stoke',
            'Swansea' : 'Swansea',
            'Swansea City' : 'Swansea', 
            'Cardiff' : 'Cardiff',
            'Cardiff City' : 'Cardiff',
            'Huddersfield' : 'Huddersfield',
            'Huddersfield Town' : 'Huddersfield',
            'Bayern Munich' : 'Bayern',
            'Dortmund' : 'Dortmund',
            'RB Lipzig' : 'RB Lipzig',
            'Union Berlin' : 'Union Berlin',
            'Frieburg' : 'Frieburg',
            'Leverkusen' : 'Leverkusen',
            'Bayer Leverkusen' : 'Leverkusen',
            'Eint Frankfurt' : 'Frankfurt',
            'Eintracht Frankfurt' : 'Frankfurt',
            'Wolfsburg' : 'Wolfsburg',
            'Mainz 05' : 'Mainz',
            "M'Gladbach" : 'Gladbach',
            'Mönchengladbach' : 'Gladbach',
            'Köln' : 'Koeln',
            'Hoffenheim' : 'Hoffenheim',
            'Werder Bremen' : 'Werder',
            'Bochum' : 'Bochum',
            'Augsburg' : 'Augsburg',
            'Stuttgart' : 'Stuttgart',
            'Schalke 04' : 'Schalke',
            'Hertha BSC' : 'Hertha',
            'Arminia' : 'Bielefeld',
            'Greuther Fürth' : 'Fuerth',
            'Düsseldorf' : 'Düsseldorf',
            'Paderborn 07' : 'Paderborn',
            'Hannover 96' : 'Hannover',
            'Nürnberg' : 'Nürnberg',
            'Hamburger SV' : 'Hamburg',
            'Heidenheim' : 'Heidenheim',
            'Inter' : 'Inter',
            'Internazionale' : 'Inter',
            'Juventus' : 'Juventus',
            'Milan' : 'Milan',
            'Roma' : 'Roma',
            'Bologna' : 'Bologna',
            'Napoli' : 'Napoli',
            'Fiorentina' : 'Fiorentina',
            'Atalanta' : 'Atalanta',
            'Monza' : 'Monza',
            'Lazio' : 'Lazio',
            'Torino' : 'Torino',
            'Frosinone' : 'Frosinone',
            'Lecce' : 'Lecce',
            'Genoa' : 'Genoa',
            'Sassuolo' : 'Sassuolo',
            'Cagliari' : 'Cagliari',
            'Udinese' : 'Udinese',
            'Empoli' : ' Empoli',
            'Hellas Verona' : 'Verona',
            'Salernitana' : 'Salernitana',
            'Spezia' : 'Spezia',
            'Cremonese' : 'Cremonese',
            'Sampdoria' : 'Sampdoria',
            'Venezia' : 'Venezia',
            'Benevento' : 'Benevento',
            'Crotone' : 'Crotone',
            'Parma' : 'Parma',
            'Brescia' : 'Brescia',
            'SPAL' : 'SPAL',
            'Chievo' : 'Chievo',
            'Paris Saint-Germain' : 'Paris SG',
            'Paris S-G' : 'Paris SG',
            'Lens' : 'Lens',
            'Marseille' : 'Marseille',
            'Rennes' : 'Rennes',
            'Lille' : 'Lille',
            'Monaco' : 'Monaco',
            'Lyon' : 'Lyon',
            'Clermont Foot' : 'Clermont',
            'Nice' : 'Nice',
            'Lorient' : 'Lorient',
            'Reims' : 'Reims',
            'Montpellier' : 'Montpellier',
            'Toulouse' : 'Toulouse',
            'Brest' : 'Brest',
            'Strasbourg' : 'Strasbourg',
            'Nantes' : 'Nantes',
            'Auxerre' : 'Auxerre',
            'Ajaccio' : 'Ajaccio',
            'Troyes' : 'Troyes',
            'Angers' : 'Angers',
            'Saint-Étienne' : 'Saint-Etienne',
            'Metz' : 'Metz',
            'Bordeaux' : 'Bordeaux',
            'Amiens' : 'Amiens',
            'Nîmes' : 'Nimes',
            'Dijon' : 'Dijon',
            'Caen' : 'Caen',
            'Guingamp' : 'Guingamp',
            'Troyes' : 'Troyes',
            'Barcelona' : 'Barcelona',
            'Real Madrid' : 'Real Madrid',
            'Atlético Madrid' : 'Atletico',
            'Real Sociedad' : 'Sociedad',
            'Villarreal' : 'Villarreal',
            'Real Betis' : 'Betis',
            'Betis' : 'Betis',
            'Osasuna' : 'Osasuna',
            'Athletic Club' : 'Bilbao',
            'Mallorca' : 'Mallorca',
            'Girona' : 'Girona',
            'Rayo Vallecano' : 'Rayo Vallecano',
            'Sevilla' : 'Sevilla',
            'Celta Vigo' : 'Celta',
            'Cádiz' : 'Cadiz',
            'Getafe' : 'Getafe',
            'Valencia' : 'Valencia',
            'Almería' : 'Almeria',
            'Valladolid' : 'Valladolid',
            'Espanyol' : 'Espanyol',
            'Granada' : 'Granada',
            'Levante' : 'Levante',
            'Alavés' : 'Alaves',
            'Elche' : 'Elche',
            'Huesca' : 'Huesca',
            'Eibar' : 'Eibar',
            'Leganés' : 'Leganes',
            'La Coruña' : 'Depor',
            'Las Palmas' : 'Las Palmas',
            'Málaga' : 'Malaga'
}

    return replacments_club_names.get(club_name, club_name)

In [32]:
# elo

def noneParser(variable, parseFunc):
    if (variable == 'None' or variable == ''):
        return None
    return parseFunc(variable)

def dateParser(variable):
    try:
        return datetime.datetime.strptime(variable, "%Y-%m-%d")
    except:
        return None

def dataFrame(dataRows):

    data = []

    for row in dataRows[1:]:
        if row == '':
            continue
        columns = row.split(',')
        entry = {}
        entry['rank'] = noneParser(columns[0], int)
        entry['club'] = noneParser(columns[1], str)
        entry['country'] = noneParser(columns[2], str)
        entry['level'] = noneParser(columns[3], int)
        entry['elo'] = noneParser(columns[4], float)
        entry['from'] = noneParser(columns[5], str)
        entry['to'] = noneParser(columns[6], str)
        data.append(entry)

    df = pd.DataFrame(data)

    return df

def get_row(match_date, df):

    filtered_df = df[(df['from'] <= match_date) & (df['to'] >= match_date)]

    elo_value = float(filtered_df['elo'])
    
    return elo_value



def getElo(club_name, match_date):

    try:
        replacement_club_name = get_replacement(club_name).lower()
        
        url_elo = "http://api.clubelo.com/"
        url = url_elo + replacement_club_name

        response = requests.get(url)
        dataRows = response.text.split('\n')
        df = dataFrame(dataRows)
        elo_ranking_vlaue = get_row(match_date, df)

        return str(elo_ranking_vlaue)

    except:
        return str(0.0)

In [33]:
# single match

def team_data(lista, nazwa, data_for_dataframe):

    pattern = r'data-stat="([^"]+)">([^<]+)'

    matches = re.findall(pattern, lista)

    for match in matches:
        data_type, value = match
        data_for_dataframe[nazwa +data_type] = value


def getMatchDate(page_source):
    date_of_match = re.findall(r'data-venue-date=([^ ]+)', page_source)
    date_of_match = date_of_match[0].replace('"', '')

    return date_of_match


def getMatchTime(page_source):
    game_time = re.findall(r'data-venue-time=([^ ]+)', page_source)
    game_time = game_time[0].replace('"', '')

    return game_time

def getClubsNames(page_source):
    teams = re.findall(r'<h1>.*Match', page_source)
    teams = teams[0].replace(' Match', '')
    teams = teams.replace('<h1>','')

    teams_names = teams.split(" vs. ")

    return teams_names


def getData(page_source):

    data_for_dataframe = {}

    data = re.findall("<tfoot>.*<\/tfoot>", page_source)

    home_team = data[0:6]
    away_team = data[6:12]

    home_team_summary = home_team[0]
    home_team_passes = home_team[1]
    home_team_pass_type = home_team[2]
    home_team_defensive = home_team[3]
    home_team_possesion = home_team[4]
    home_team_miscellaneous = home_team[5]

    away_team_summary = away_team[0]
    away_team_passes = away_team[1]
    away_team_pass_type = away_team[2]
    away_team_defensive = away_team[3]
    away_team_possesion = away_team[4]
    away_team_miscellaneous = away_team[5]

    team_data(home_team_summary, 'home_team_summary_', data_for_dataframe)
    team_data(home_team_passes, 'home_team_passes_', data_for_dataframe)
    team_data(home_team_pass_type, 'home_team_pass_type_', data_for_dataframe)
    team_data(home_team_defensive, 'home_team_defensive_', data_for_dataframe)
    team_data(home_team_possesion, 'home_team_possesion_', data_for_dataframe)
    team_data(home_team_miscellaneous, 'home_team_miscellaneous_', data_for_dataframe)

    team_data(away_team_summary, 'away_team_summary_', data_for_dataframe)
    team_data(away_team_passes, 'away_team_passes_', data_for_dataframe)
    team_data(away_team_pass_type, 'away_team_pass_type_', data_for_dataframe)
    team_data(away_team_defensive, 'away_team_defensive_', data_for_dataframe)
    team_data(away_team_possesion, 'away_team_possesion_', data_for_dataframe)
    team_data(away_team_miscellaneous, 'away_team_miscellaneous_', data_for_dataframe)

    df = pd.DataFrame.from_dict(data_for_dataframe, orient ='index')

    df = df.T

    df['game_date'] = getMatchDate(page_source)
    df['game_time'] = getMatchTime(page_source)

    teams_names = getClubsNames(page_source)

    df['home_team'] = teams_names[0]
    df['away_team'] = teams_names[1]

    return df


def getDataFromPage(url):

    op = webdriver.ChromeOptions()
    op.add_argument('headless')

    driver = webdriver.Chrome(options=op)
    driver.get(url)

    page_source = driver.page_source

    driver.quit()

    return page_source




def eloData(df):

    home_team = df['home_team'][0]
    away_team = df['away_team'][0]

    home_team_name_for_elo = get_replacement(home_team).lower().replace(" ", "")
    away_team_name_for_elo = get_replacement(away_team).lower().replace(" ", "")

    df['elo_home_team'] = getElo(get_replacement(home_team_name_for_elo), str(df['game_date'][0]))
    df['elo_away_team'] = getElo(get_replacement(away_team_name_for_elo), str(df['game_date'][0]))

    return df


def singleMatchData(url):

    data = getDataFromPage(url)
    dataframe = getData(data)

    dataframe_with_elo = eloData(dataframe)

    return dataframe_with_elo

In [34]:
# mongo

from credencials_mongo import user_name, password

def uploadMongo(data_to_mongo):

    client_link = "mongodb+srv://"+ user_name + ":" + password + "@football.cvyivdz.mongodb.net/"

    client = pymongo.MongoClient(client_link)

    db = client.football
    collection = db.predictions_germany

    result = collection.insert_one(data_to_mongo)

    print("Inserted document ID:", result.inserted_id)

    client.close()

In [35]:
# main

def parseData(dane):
    dict_to_mongo = dane.to_dict('records')
    dict_to_mongo = dict_to_mongo[0]

    return dict_to_mongo


main_urls = ['https://fbref.com/en/comps/12/2017-2018/schedule/2017-2018-La-Liga-Scores-and-Fixtures']  


urls = ['https://fbref.com//en/matches/65a76db9/Stuttgart-Augsburg-March-19-2022-Bundesliga',
'https://fbref.com//en/matches/df2c244b/Hertha-BSC-Hoffenheim-March-19-2022-Bundesliga',
'https://fbref.com//en/matches/e3e5a297/Mainz-05-Arminia-March-19-2022-Bundesliga',
'https://fbref.com//en/matches/f0d8f18c/Greuther-Furth-Freiburg-March-19-2022-Bundesliga',
'https://fbref.com//en/matches/8a7edc58/Bayern-Munich-Union-Berlin-March-19-2022-Bundesliga',
'https://fbref.com//en/matches/5245b27f/RB-Leipzig-Eintracht-Frankfurt-March-20-2022-Bundesliga',
'https://fbref.com//en/matches/6e3b2c49/Wolfsburg-Bayer-Leverkusen-March-20-2022-Bundesliga',
'https://fbref.com//en/matches/4e9be983/Koln-Dortmund-March-20-2022-Bundesliga',
'https://fbref.com//en/matches/842c282d/Union-Berlin-Koln-April-1-2022-Bundesliga',
'https://fbref.com//en/matches/3cf9ef0b/Arminia-Stuttgart-April-2-2022-Bundesliga',
'https://fbref.com//en/matches/3d271a87/Hoffenheim-Bochum-April-2-2022-Bundesliga',
'https://fbref.com//en/matches/5be07425/Bayer-Leverkusen-Hertha-BSC-April-2-2022-Bundesliga',
'https://fbref.com//en/matches/84dfca9b/Eintracht-Frankfurt-Greuther-Furth-April-2-2022-Bundesliga',
'https://fbref.com//en/matches/c49cc77e/Freiburg-Bayern-Munich-April-2-2022-Bundesliga',
'https://fbref.com//en/matches/142e57da/Dortmund-RB-Leipzig-April-2-2022-Bundesliga',
'https://fbref.com//en/matches/940fc2a6/Augsburg-Wolfsburg-April-3-2022-Bundesliga',
'https://fbref.com//en/matches/2665736f/Monchengladbach-Mainz-05-April-3-2022-Bundesliga',
'https://fbref.com//en/matches/d614186f/Augsburg-Mainz-05-April-6-2022-Bundesliga',
'https://fbref.com//en/matches/19e1bbea/Stuttgart-Dortmund-April-8-2022-Bundesliga',
'https://fbref.com//en/matches/32edb6b6/Bayern-Munich-Augsburg-April-9-2022-Bundesliga',
'https://fbref.com//en/matches/9642cb8e/Greuther-Furth-Monchengladbach-April-9-2022-Bundesliga',
'https://fbref.com//en/matches/b2468a52/Koln-Mainz-05-April-9-2022-Bundesliga',
'https://fbref.com//en/matches/fa852ccf/Wolfsburg-Arminia-April-9-2022-Bundesliga',
'https://fbref.com//en/matches/04851f34/Hertha-BSC-Union-Berlin-April-9-2022-Bundesliga',
'https://fbref.com//en/matches/25412678/Bochum-Bayer-Leverkusen-April-10-2022-Bundesliga',
'https://fbref.com//en/matches/e7491da8/Eintracht-Frankfurt-Freiburg-April-10-2022-Bundesliga',
'https://fbref.com//en/matches/b6aa7132/RB-Leipzig-Hoffenheim-April-10-2022-Bundesliga',
'https://fbref.com//en/matches/2692dcc4/Dortmund-Wolfsburg-April-16-2022-Bundesliga',
'https://fbref.com//en/matches/7eeac6fb/Mainz-05-Stuttgart-April-16-2022-Bundesliga',
'https://fbref.com//en/matches/d568b6dd/Freiburg-Bochum-April-16-2022-Bundesliga',
'https://fbref.com//en/matches/ea17a6ff/Augsburg-Hertha-BSC-April-16-2022-Bundesliga',
'https://fbref.com//en/matches/4e0481f4/Monchengladbach-Koln-April-16-2022-Bundesliga',
'https://fbref.com//en/matches/af8c9180/Arminia-Bayern-Munich-April-17-2022-Bundesliga',
'https://fbref.com//en/matches/1155fe35/Union-Berlin-Eintracht-Frankfurt-April-17-2022-Bundesliga',
'https://fbref.com//en/matches/bab6d1dd/Hoffenheim-Greuther-Furth-April-17-2022-Bundesliga',
'https://fbref.com//en/matches/d4cc9ae4/Bayer-Leverkusen-RB-Leipzig-April-17-2022-Bundesliga',
'https://fbref.com//en/matches/9f31658e/Wolfsburg-Mainz-05-April-22-2022-Bundesliga',
'https://fbref.com//en/matches/10162268/Eintracht-Frankfurt-Hoffenheim-April-23-2022-Bundesliga',
'https://fbref.com//en/matches/63a9230b/Koln-Arminia-April-23-2022-Bundesliga',
'https://fbref.com//en/matches/72082975/Freiburg-Monchengladbach-April-23-2022-Bundesliga',
'https://fbref.com//en/matches/aa4dd599/RB-Leipzig-Union-Berlin-April-23-2022-Bundesliga',
'https://fbref.com//en/matches/da549a9d/Greuther-Furth-Bayer-Leverkusen-April-23-2022-Bundesliga',
'https://fbref.com//en/matches/8c04e2e7/Der-Klassiker-Bayern-Munich-Dortmund-April-23-2022-Bundesliga',
'https://fbref.com//en/matches/29085f79/Bochum-Augsburg-April-24-2022-Bundesliga',
'https://fbref.com//en/matches/62595894/Hertha-BSC-Stuttgart-April-24-2022-Bundesliga',
'https://fbref.com//en/matches/621664f9/Union-Berlin-Greuther-Furth-April-29-2022-Bundesliga',
'https://fbref.com//en/matches/257ad808/Dortmund-Bochum-April-30-2022-Bundesliga',
'https://fbref.com//en/matches/7d4c3f41/Stuttgart-Wolfsburg-April-30-2022-Bundesliga',
'https://fbref.com//en/matches/907cd34d/Arminia-Hertha-BSC-April-30-2022-Bundesliga',
'https://fbref.com//en/matches/9d63dd8a/Augsburg-Koln-April-30-2022-Bundesliga',
'https://fbref.com//en/matches/f81e9135/Mainz-05-Bayern-Munich-April-30-2022-Bundesliga',
'https://fbref.com//en/matches/c7076c62/Hoffenheim-Freiburg-April-30-2022-Bundesliga',
'https://fbref.com//en/matches/84b03670/Monchengladbach-RB-Leipzig-May-2-2022-Bundesliga',
'https://fbref.com//en/matches/a2953956/Bayer-Leverkusen-Eintracht-Frankfurt-May-2-2022-Bundesliga',
'https://fbref.com//en/matches/73983b19/Bochum-Arminia-May-6-2022-Bundesliga',
'https://fbref.com//en/matches/389a34e9/Hoffenheim-Bayer-Leverkusen-May-7-2022-Bundesliga',
'https://fbref.com//en/matches/7e770cf7/Koln-Wolfsburg-May-7-2022-Bundesliga',
'https://fbref.com//en/matches/d1f394f3/Greuther-Furth-Dortmund-May-7-2022-Bundesliga',
'https://fbref.com//en/matches/d6c40f7a/Freiburg-Union-Berlin-May-7-2022-Bundesliga',
'https://fbref.com//en/matches/cf7018f2/Hertha-BSC-Mainz-05-May-7-2022-Bundesliga',
'https://fbref.com//en/matches/b9b0b83a/Eintracht-Frankfurt-Monchengladbach-May-8-2022-Bundesliga',
'https://fbref.com//en/matches/f20efaaf/Bayern-Munich-Stuttgart-May-8-2022-Bundesliga',
'https://fbref.com//en/matches/8ac13dab/RB-Leipzig-Augsburg-May-8-2022-Bundesliga',
'https://fbref.com//en/matches/31d4f881/Wolfsburg-Bayern-Munich-May-14-2022-Bundesliga',
'https://fbref.com//en/matches/47e4a1a7/Stuttgart-Koln-May-14-2022-Bundesliga',
'https://fbref.com//en/matches/79ec8323/Bayer-Leverkusen-Freiburg-May-14-2022-Bundesliga',
'https://fbref.com//en/matches/86428aab/Dortmund-Hertha-BSC-May-14-2022-Bundesliga',
'https://fbref.com//en/matches/976cf786/Mainz-05-Eintracht-Frankfurt-May-14-2022-Bundesliga',
'https://fbref.com//en/matches/b8402ce1/Monchengladbach-Hoffenheim-May-14-2022-Bundesliga',
'https://fbref.com//en/matches/c8cef118/Union-Berlin-Bochum-May-14-2022-Bundesliga',
'https://fbref.com//en/matches/e0a9b42f/Augsburg-Greuther-Furth-May-14-2022-Bundesliga',
'https://fbref.com//en/matches/ed07da7a/Arminia-RB-Leipzig-May-14-2022-Bundesliga']


# for main_url in main_urls:
#     print(main_url)
#     urls = matchesLinks(main_url)

for url in urls:
    if url in {'https://fbref.com//en/matches/c34bbc21/Bochum-Monchengladbach-March-18-2022-Bundesliga',
                'https://fbref.com//en/matches/e0a20cfe/Hellas-Verona-Roma-September-19-2020-Serie-A'}:
        print(url + ' NO!')
    else:
        print(url)
        data = singleMatchData(url)
        dict_to_mongo = parseData(data)

        uploadMongo(dict_to_mongo)

https://fbref.com//en/matches/65a76db9/Stuttgart-Augsburg-March-19-2022-Bundesliga
Inserted document ID: 657c81f128512af54cd398dc
https://fbref.com//en/matches/df2c244b/Hertha-BSC-Hoffenheim-March-19-2022-Bundesliga
Inserted document ID: 657c81f928512af54cd398de
https://fbref.com//en/matches/e3e5a297/Mainz-05-Arminia-March-19-2022-Bundesliga
Inserted document ID: 657c820128512af54cd398e0
https://fbref.com//en/matches/f0d8f18c/Greuther-Furth-Freiburg-March-19-2022-Bundesliga
Inserted document ID: 657c820a28512af54cd398e2
https://fbref.com//en/matches/8a7edc58/Bayern-Munich-Union-Berlin-March-19-2022-Bundesliga
Inserted document ID: 657c821128512af54cd398e4
https://fbref.com//en/matches/5245b27f/RB-Leipzig-Eintracht-Frankfurt-March-20-2022-Bundesliga
Inserted document ID: 657c821928512af54cd398e6
https://fbref.com//en/matches/6e3b2c49/Wolfsburg-Bayer-Leverkusen-March-20-2022-Bundesliga
Inserted document ID: 657c822128512af54cd398e8
https://fbref.com//en/matches/4e9be983/Koln-Dortmund-Mar